# Google Drive Mount

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/final_project/김예지')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

### Unzip file

In [ ]:
!pwd

/content/drive/MyDrive/final_project/김예지


In [ ]:
import zipfile
import os

input_path = 'data/kakao_origin.zip'
output_dir = 'data/origin'

os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

압축 해제 경로: data/origin


# Image Tiling

In [ ]:
import numpy as np
import cv2
from skimage.util import view_as_windows
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm

img_dir = 'data/origin/'
file_names = [file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
len(file_names)

3533

In [ ]:
for img_file in tqdm(file_names):
  img_path = img_dir + img_file

  img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  tile_size = 512
  stride = 256

  tiles = view_as_windows(img, (tile_size, tile_size, 3), step=stride)
  tiles = tiles.squeeze(axis=2)

  num_rows, num_cols, tile_height, tile_width, num_channels = tiles.shape

  tile_list = []
  for i in range(num_rows):
      for j in range(num_cols):
          tile = tiles[i, j]
          tile_list.append(tile)

  # 타일링 경로
  tiling_dir = 'data/tiling'
  os.makedirs(tiling_dir, exist_ok=True)

  for i in range(len(tile_list)):
    # 학습용 ----------------------------
    # img_array = tile_list[i]
    # img = Image.fromarray(img_array)
    # fN = img_file.split('_')
    # fileName = tiling_dir + "/" + str(fN[2]) + '_' + str(fN[3]) + '_' + str(i+1)  + '.png'
    # img.save(fileName)

    # 테스트용(카카오맵) --------------------------------
    img_array = tile_list[i]
    img = Image.fromarray(img_array)
    fN = img_file.split('.png')[0]
    fileName = tiling_dir + '/' + fN + '_' + str(i+1) + '.png'
    img.save(fileName)

100%|██████████| 3533/3533 [1:03:40<00:00,  1.08s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/final_project/김예지/data/tiling/'
file_names = [file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
len(file_names)

31797

In [ ]:
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

tiling_dir =  '/content/drive/MyDrive/final_project/김예지/data/tiling'
output_dir = "/content/drive/MyDrive/final_project/김예지/data/tiling_zip"
os.makedirs(output_dir, exist_ok=True)

file_list = sorted([os.path.join(tiling_dir, f) for f in os.listdir(tiling_dir) if f.endswith('.png')])

num_files = len(file_list)
split_size = num_files // 15
file_groups = [file_list[i * split_size : (i + 1) * split_size] for i in range(14)]
file_groups.append(file_list[14 * split_size:])

def compress_files(index, group):
    zip_filename = os.path.join(output_dir, f'tiling_part{index+1}.zip')
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in group:
            zipf.write(file, os.path.basename(file))
    print(f"✅ {zip_filename} 생성 완료!")

with ThreadPoolExecutor(max_workers = 4) as executor:
    list(tqdm(executor.map(compress_files, range(15), file_groups), total=15))

print("📂 모든 압축 파일이 Google Drive에 저장되었습니다.")

  0%|          | 0/15 [00:00<?, ?it/s]

✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part3.zip 생성 완료!
✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part4.zip 생성 완료!
✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part2.zip 생성 완료!


  7%|▋         | 1/15 [13:06<3:03:35, 786.81s/it]

✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part1.zip 생성 완료!


 33%|███▎      | 5/15 [14:40<22:50, 137.03s/it]  

...

✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part13.zip 생성 완료!


 93%|█████████▎| 14/15 [17:54<00:30, 30.16s/it]

✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part14.zip 생성 완료!


100%|██████████| 15/15 [17:56<00:00, 71.79s/it]

✅ /content/drive/MyDrive/final_project/김예지/data/tiling_zip/tiling_part15.zip 생성 완료!
📂 모든 압축 파일이 Google Drive에 저장되었습니다.


# Real-ESRGAN

## git clone

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git

fatal: destination path 'Real-ESRGAN' already exists and is not an empty directory.


In [ ]:
%cd Real-ESRGAN

/content/drive/MyDrive/fin_project/Real-ESRGAN


In [ ]:
!pip install -r requirements.txt
!pip install torchvision==0.16.2
!python setup.py develop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.

## upscaling

In [ ]:
%cd ..
!pwd

/content/drive/MyDrive/fin_project
/content/drive/MyDrive/fin_project


### part1

In [ ]:
import zipfile
import os

part = 'tiling_part1'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))


압축 해제 경로: /content/tiling_part1


In [ ]:

# os.makedirs(upscaling_dir, exist_ok=True)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:15:32<00:00,  5.06s/it]


### part2

In [ ]:
import zipfile
import os

part = 'tiling_part2'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part2
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:20:49<00:00,  5.20s/it]


### part3

In [ ]:
import zipfile
import os

part = 'tiling_part3'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part3
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:23:19<00:00,  5.27s/it]


### part4

In [ ]:
import zipfile
import os

part = 'tiling_part4'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part4
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:27:38<00:00,  5.38s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

9234


### part5

In [ ]:
import zipfile
import os

part = 'tiling_part5'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part5
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:35:35<00:00,  5.58s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

11540


### part6

In [ ]:
import zipfile
import os

part = 'tiling_part6'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part6
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:39:43<00:00,  5.69s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

13813


### part7

In [ ]:
import zipfile
import os

part = 'tiling_part7'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part7
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names[1783:]), total=len(file_names[1783:])))

100%|██████████| 534/534 [48:06<00:00,  5.40s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

16098


### part8

In [ ]:
import zipfile
import os

part = 'tiling_part8'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part8
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:21:51<00:00,  5.23s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

18413


### part9

In [ ]:
import zipfile
import os

part = 'tiling_part9'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part9
2317


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2317/2317 [3:27:43<00:00,  5.38s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

20722


### part10

In [ ]:
import zipfile
import os

part = 'tiling_part10'
input_path = 'data/tiling_zip/'+ part +'.zip'
output_dir = '/content/' + part

os.makedirs(output_dir, exist_ok=True)

# ZIP 파일 압축 해제
with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"압축 해제 경로: {output_dir}")

img_dir = '/content/' + part + '/'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

압축 해제 경로: /content/tiling_part10
2322


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
from tqdm import tqdm

def upscale_image(file_name):
    command = f"CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {file_name} -o {upscaling_dir} --outscale 1"
    subprocess.run(command, shell=True)

upscaling_dir = 'data/upscaling'

with ThreadPoolExecutor(max_workers=7) as executor:
    list(tqdm(executor.map(upscale_image, file_names), total=len(file_names)))

100%|██████████| 2322/2322 [3:35:25<00:00,  5.57s/it]


In [ ]:
img_dir = '/content/drive/MyDrive/fin_project/data/upscaling'
file_names = [img_dir + file for file in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, file))]
print(len(file_names))

23029


In [ ]:
import shutil

input_folder = "/content/drive/MyDrive/fin_project/data/upscaling"
output_zip = "/content/drive/My Drive/fin_project/data/origin.zip"

shutil.make_archive(output_zip.replace(".zip", ""), 'zip', root_dir=input_folder)